In [1]:
import os
# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

In [2]:
MODEL = 'gpt-4o-mini'

In [3]:
from openai import OpenAI
client = OpenAI()

def recommender_chat_bot(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You are a chatbot system \
            and recommender for products about tech to a diverse group of users."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content

In [4]:
def generate_data_with_chatgpt(data_example):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You generate examples of tech products."},
                  {"role": "user", "content": data_example}]
    )
    
    return response.choices[0].message.content


tech_products_prompt = "Create 20 examples of tech products\
    with their descriptions as a list of bullet points"

data_tech = generate_data_with_chatgpt(tech_products_prompt)
data_tech

'Sure! Here’s a list of 20 tech products along with their descriptions:\n\n1. **Smartphone-X Pro**\n   - A high-performance smartphone featuring a 108MP camera, 5G connectivity, and a 6.7-inch OLED display for vibrant visuals.\n\n2. **EcoCharge Solar Power Bank**\n   - A portable power bank that harnesses solar energy, equipped with dual USB ports and a rugged design for outdoor adventures.\n\n3. **Smart Home Hub**\n   - A central device that connects and manages all smart appliances, providing voice control through AI and integrating with popular home automation systems.\n\n4. **Wireless Noise-Canceling Headphones**\n   - Over-ear headphones that offer immersive sound quality with customizable noise-canceling settings and a battery life of up to 30 hours.\n\n5. **AI-Powered Personal Assistant**\n   - A voice-activated device that manages schedules, controls home devices, and answers questions using advanced machine learning algorithms.\n\n6. **4K Ultra HD Streaming Stick**\n   - A com

In [5]:
from IPython.display import Markdown
Markdown(data_tech)

Sure! Here’s a list of 20 tech products along with their descriptions:

1. **Smartphone-X Pro**
   - A high-performance smartphone featuring a 108MP camera, 5G connectivity, and a 6.7-inch OLED display for vibrant visuals.

2. **EcoCharge Solar Power Bank**
   - A portable power bank that harnesses solar energy, equipped with dual USB ports and a rugged design for outdoor adventures.

3. **Smart Home Hub**
   - A central device that connects and manages all smart appliances, providing voice control through AI and integrating with popular home automation systems.

4. **Wireless Noise-Canceling Headphones**
   - Over-ear headphones that offer immersive sound quality with customizable noise-canceling settings and a battery life of up to 30 hours.

5. **AI-Powered Personal Assistant**
   - A voice-activated device that manages schedules, controls home devices, and answers questions using advanced machine learning algorithms.

6. **4K Ultra HD Streaming Stick**
   - A compact streaming device enabling access to various streaming platforms in 4K quality, with voice search and a user-friendly interface.

7. **Fitness Tracking Smartwatch**
   - A sleek smartwatch that monitors heart rate, activity levels, sleep patterns, and provides GPS tracking for workouts, all synced with a mobile app.

8. **Virtual Reality Headset**
   - An immersive VR headset featuring high-resolution displays and motion tracking, offering a wide range of gaming and interactive experiences.

9. **Smart Thermostat**
   - A programmable thermostat that learns your heating and cooling preferences, optimizing energy usage and controllable via a smartphone app.

10. **Gaming Laptop with RTX GPU**
    - A powerful gaming laptop equipped with the latest NVIDIA RTX graphics card, delivering high-definition gaming experiences and fast processing speeds.

11. **Smart Security Camera**
    - A Wi-Fi-enabled security camera featuring motion detection, night vision, and two-way audio, allowing users to monitor their homes from anywhere.

12. **Portable Bluetooth Speaker**
    - A waterproof Bluetooth speaker with powerful acoustics, offering up to 12 hours of battery life for outdoor music enjoyment.

13. **Augmented Reality Glasses**
    - Lightweight AR glasses providing overlay information on the real world, compatible with various applications for gaming, navigation, and education.

14. **Wireless Charging Pad**
    - A sleek and efficient charging pad that supports multiple devices simultaneously with fast charging capabilities for compatible smartphones.

15. **AI Language Translator Device**
    - A handheld device that provides real-time language translation, supporting multiple languages and ideal for travelers and business professionals alike.

16. **Smart Refrigerator**
    - An advanced refrigerator with a touch-screen interface, equipped with internal cameras to manage groceries remotely and suggest recipes.

17. **Robot Vacuum Cleaner**
    - An automated vacuum cleaner that uses AI for efficient cleaning patterns, equipped with Wi-Fi connectivity for scheduled cleaning via a mobile app.

18. **Multi-Device Wireless Keyboard**
    - A versatile keyboard that can seamlessly switch between connected devices, featuring ergonomic design and customizable shortcuts.

19. **Digital Whiteboard**
    - An interactive whiteboard ideal for classrooms and meetings, allowing users to write, draw, and share content digitally with integrated connectivity options.

20. **Smart Plant Monitor**
    - A device that monitors soil moisture, sunlight, and temperature, providing care recommendations through a connected app for gardening enthusiasts.

Feel free to ask for more or any specific details about these products!

In [6]:
def create_costumer_profile(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You create a list of customer \
            profiles based on input."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content


create_costumer_profile = create_costumer_profile("Create 10 customer profiles \
    for users with different backgrounds and preferences.")

Markdown(create_costumer_profile)

Here are 10 diverse customer profiles based on different backgrounds and preferences:

1. **Profile Name:** Tech-Savvy Millennial  
   - **Age:** 28  
   - **Occupation:** Software Engineer  
   - **Location:** San Francisco, CA  
   - **Interests:** Latest tech gadgets, online gaming, digital art  
   - **Purchasing Preferences:** Prefers online shopping, values user reviews and product testing videos; frequent buyer of subscription services for streaming and software.  

2. **Profile Name:** Eco-Conscious Parent  
   - **Age:** 35  
   - **Occupation:** Elementary School Teacher  
   - **Location:** Portland, OR  
   - **Interests:** Sustainable living, outdoor activities, family wellness  
   - **Purchasing Preferences:** Looks for eco-friendly products, interested in organic and local goods, often purchases educational toys for children.  

3. **Profile Name:** Retired Traveler  
   - **Age:** 65  
   - **Occupation:** Retired (Former Marketing Executive)  
   - **Location:** Miami, FL  
   - **Interests:** Traveling, gardening, volunteering  
   - **Purchasing Preferences:** Prefers high-quality travel gear, takes advantage of senior discounts, enjoys booking through travel agencies.  

4. **Profile Name:** Health and Fitness Enthusiast  
   - **Age:** 30  
   - **Occupation:** Personal Trainer  
   - **Location:** New York, NY  
   - **Interests:** Fitness classes, nutrition, wellness coaching  
   - **Purchasing Preferences:** Frequently purchases athletic apparel and supplements, prefers brands that promote a healthy lifestyle, subscribes to fitness apps and services.  

5. **Profile Name:** Fashion-Forward Student  
   - **Age:** 21  
   - **Occupation:** College Student (Fashion Design)  
   - **Location:** Los Angeles, CA  
   - **Interests:** Fashion trends, social media, thrifting  
   - **Purchasing Preferences:** Enjoys shopping for trendy, affordable clothes, often uses social media influencers for inspiration, values sustainability in fashion.  

6. **Profile Name:** Busy Professional  
   - **Age:** 40  
   - **Occupation:** Corporate Finance Manager  
   - **Location:** Chicago, IL  
   - **Interests:** Networking, gourmet cooking, fine dining  
   - **Purchasing Preferences:** Prefers convenient meal kits and quick service restaurants, regularly dines out, invests in high-end kitchen appliances.  

7. **Profile Name:** Indie Music Aficionado  
   - **Age:** 27  
   - **Occupation:** Graphic Designer  
   - **Location:** Seattle, WA  
   - **Interests:** Live music, art galleries, local craft beers  
   - **Purchasing Preferences:** Enjoys vinyl records, supports local artists and labels, frequently attends music festivals and concerts.  

8. **Profile Name:** Family-Focused Homebody  
   - **Age:** 45  
   - **Occupation:** Stay-at-Home Mom  
   - **Location:** Suburban Texas  
   - **Interests:** Cooking, DIY projects, family-oriented activities  
   - **Purchasing Preferences:** Looks for bulk-buy options for groceries, enjoys home improvement and craft supplies, often purchases family board games and outdoor equipment.  

9. **Profile Name:** Adventurous Single  
   - **Age:** 32  
   - **Occupation:** Outdoor Guide  
   - **Location:** Colorado  
   - **Interests:** Hiking, camping, rock climbing  
   - **Purchasing Preferences:** Invests in durable outdoor gear, seeks experiential purchases (such as guided tours), frequently looks for travel deals to wilderness locations.  

10. **Profile Name:** Tech Industry Executive  
   - **Age:** 50  
   - **Occupation:** Chief Technology Officer  
   - **Location:** Austin, TX  
   - **Interests:** Innovations in technology, philanthropy, fine arts  
   - **Purchasing Preferences:** Buys premium tech products and services, values corporate social responsibility, enjoys fine dining experiences with colleagues and industry peers.  

These profiles illustrate a variety of lifestyles, interests, and purchasing decisions, showcasing how customer behaviors can differ widely based on background and preferences.

In [7]:
profile_10 = """
Customer Profile 10: Carlos Ramirez
Age: 38
Occupation: Restaurant Owner
Location: San Diego, CA
Interests: Culinary arts, wine tasting, fitness
Shopping Preferences: Seeks gourmet cooking supplies and health-conscious food options. Enjoys premium quality products.
Spending Behavior: High spender, especially in gourmet and culinary-related categories.
These profiles illustrate the complexity of customer preferences, behaviors, and demographics.
"""


def recommend_product(profile):
    prompt_to_recommender = f"Recommend a tech product from this list ONLY:\n\n{data_tech} \
        for this customer profile: {profile}.\
        Output ONLY A SIMPLE MESSAGE REFERENCING THE CUSTOMER AND THE recommendations."
    
    recommendation = recommender_chat_bot(prompt_to_recommender)
    
    return recommendation


product_recommendation = recommend_product(profile_10)

Markdown(product_recommendation)

Hi Carlos! Based on your interest in culinary arts and premium quality products, I recommend the **Smart Refrigerator**. It features a touch-screen interface and internal cameras to manage your groceries and suggest recipes, making it perfect for a restaurant owner like you. Enjoy your cooking!

In [8]:
data_tech

'Sure! Here’s a list of 20 tech products along with their descriptions:\n\n1. **Smartphone-X Pro**\n   - A high-performance smartphone featuring a 108MP camera, 5G connectivity, and a 6.7-inch OLED display for vibrant visuals.\n\n2. **EcoCharge Solar Power Bank**\n   - A portable power bank that harnesses solar energy, equipped with dual USB ports and a rugged design for outdoor adventures.\n\n3. **Smart Home Hub**\n   - A central device that connects and manages all smart appliances, providing voice control through AI and integrating with popular home automation systems.\n\n4. **Wireless Noise-Canceling Headphones**\n   - Over-ear headphones that offer immersive sound quality with customizable noise-canceling settings and a battery life of up to 30 hours.\n\n5. **AI-Powered Personal Assistant**\n   - A voice-activated device that manages schedules, controls home devices, and answers questions using advanced machine learning algorithms.\n\n6. **4K Ultra HD Streaming Stick**\n   - A com

In [9]:
from pydantic import BaseModel, Field
from typing import List


class TechProduct(BaseModel):
    names: List[str] = Field(..., title="All the product names", description="The titles of the products in question")
    descriptions: List[str] = Field(..., title="All the product descriptions", description="A complete list of descriptions of the products extracted from the source.")
    
def extract_tech_product_data(raw_tech_data: str):
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {'role': 'system', 'content': 'You are an extraction engine for tech products, \
                you extract the following fields:\
                1. name - list of all names of products in the input\
                2. description - list of all descritionps in the products\
                    Make sure to extract all the names and descriptions from the input.'},
        ],
        response_format=TechProduct,
        temperature=0.0
    )
    
    return response.choices[0].message

tech_data_structured = extract_tech_product_data(f'Extract all the names and descriptions from this raw data: \n\n {data_tech}')
tech_data_structured

ParsedChatCompletionMessage[TechProduct](content='{"names":["iPhone 15","Samsung Galaxy S23","Google Pixel 8","MacBook Pro 2023","Dell XPS 13"],"descriptions":["The latest iPhone model featuring an A17 Bionic chip, improved camera system, and longer battery life.","Samsung\'s flagship smartphone with a dynamic AMOLED display, advanced camera features, and 5G connectivity.","Google\'s newest Pixel phone offering a pure Android experience, enhanced AI capabilities, and a high-quality camera.","Apple\'s newest laptop with the M3 chip, offering improved performance, longer battery life, and a stunning Retina display.","Dell\'s premium ultrabook with a sleek design, powerful Intel processors, and a vibrant InfinityEdge display."]}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=TechProduct(names=['iPhone 15', 'Samsung Galaxy S23', 'Google Pixel 8', 'MacBook Pro 2023', 'Dell XPS 13'], descriptions=['The latest iPhone model featuring an A17 Bionic chip,

In [10]:
tech_data_structured.parsed.names

['iPhone 15',
 'Samsung Galaxy S23',
 'Google Pixel 8',
 'MacBook Pro 2023',
 'Dell XPS 13']

In [11]:
tech_data_structured.parsed.descriptions

['The latest iPhone model featuring an A17 Bionic chip, improved camera system, and longer battery life.',
 "Samsung's flagship smartphone with a dynamic AMOLED display, advanced camera features, and 5G connectivity.",
 "Google's newest Pixel phone offering a pure Android experience, enhanced AI capabilities, and a high-quality camera.",
 "Apple's newest laptop with the M3 chip, offering improved performance, longer battery life, and a stunning Retina display.",
 "Dell's premium ultrabook with a sleek design, powerful Intel processors, and a vibrant InfinityEdge display."]

In [12]:
def create_diagrams_with_AI(user_query):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": "You create mermaid diagrams with AI."},
                  {"role": "user", "content": user_query}]
    )
    
    return response.choices[0].message.content

output_diagram_raw = create_diagrams_with_AI("Create a diagram of a flowchart that shows the process of a customer \
    buying a product from an e-commerce website.")

Markdown(output_diagram_raw)

Sure! Below is a textual representation of a flowchart for the process of a customer buying a product from an e-commerce website, using Mermaid syntax.

```mermaid
flowchart TD
    A[Start] --> B[Visit E-commerce Website]
    B --> C[Browse Products]
    C --> D{Select a Product?}
    D -- Yes --> E[Add Product to Cart]
    D -- No --> C
    E --> F[View Cart]
    F --> G{Proceed to Checkout?}
    G -- Yes --> H[Enter Shipping Information]
    G -- No --> C
    H --> I[Choose Payment Method]
    I --> J[Enter Payment Information]
    J --> K{Confirm Purchase?}
    K -- Yes --> L[Process Order]
    K -- No --> F
    L --> M[Receive Confirmation Email]
    M --> N[End]
```

You can use this code in a Markdown editor that supports Mermaid to visualize the flowchart.

In [13]:
class MermaidDiagram(BaseModel):
    diagram: str = Field(..., title="Mermaid Diagram", description="The mermaid diagram as a properly formatted markdown ready to visualize.")

def create_diagrams_with_AI(user_query):
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "system", "content": "You create mermaid diagrams with AI ready to be visualized in markdown."},
                  {"role": "user", "content": user_query}],
        response_format=MermaidDiagram,
        temperature=0.0
    )
    
    return response.choices[0].message.parsed.diagram

output_diagram_raw = create_diagrams_with_AI("Create a diagram of a flowchart that shows the process of a customer \
    buying a product from an e-commerce website.")

Markdown(output_diagram_raw)

```mermaid
flowchart TD
    A[Start] --> B[Visit E-commerce Website]
    B --> C[Browse Products]
    C --> D[Select Product]
    D --> E[Add to Cart]
    E --> F[View Cart]
    F --> G[Proceed to Checkout]
    G --> H[Enter Shipping Information]
    H --> I[Enter Payment Information]
    I --> J[Review Order]
    J --> K[Place Order]
    K --> L[Receive Confirmation]
    L --> M[End]
```